In [1]:
import h5py
import tensorflow as tf
import tflearn

In [4]:
num_cells = 512
num_layers = 1
timesteps = 20
learning_rate = 0.0001
batch_size=256

net = tflearn.input_data(shape=[None, timesteps, 4096], name='input')
net = tflearn.batch_normalization(net, name='normalization')
for i in range(num_layers):
    net = tflearn.layers.recurrent.lstm(net, num_cells, return_seq=True, name='lstm_{}'.format(i+1))

net = tflearn.time_distributed(net, tflearn.fully_connected, [202, 'softmax'])

In [19]:
target = tf.placeholder(tf.int32, shape=(None, 20))
target_ = SimpleSparseTensorFrom(target)
loss = tf.nn.ctc_loss(net, target_, sequence_length=[timesteps]*batch_size, time_major=False)
optimizer = tflearn.optimizers.RMSProp(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

TypeError: 'Tensor' object is not iterable.

In [23]:
def SimpleSparseTensorFrom(x):
    """Create a very simple SparseTensor with dimensions (batch, time).
    Args:
    x: a list of lists of type int
    Returns:
    x_ix and x_val, the indices and values of the SparseTensor<2>.
    """
    x_ix = []
    x_val = []
    batch_size = x.shape
    for batch_i, batch in enumerate(x):
        for time, val in enumerate(batch):
            x_ix.append([batch_i, time])
            x_val.append(val)
    x_shape = [len(x), np.asarray(x_ix).max(0)[1]+1]
    x_ix = tf.constant(x_ix, tf.int64)
    x_val = tf.constant(x_val, tf.int32)
    x_shape = tf.constant(x_shape, tf.int64)

    return x_ix, x_val, x_shape

In [ ]:
target_ind = tf.placeholder(tf.int64, shape=(None, timesteps))
target_val = tf.placeholder(tf.int64, shape=(None,))
target_shape = tf.placeholder(tf.int64, shape=())

In [9]:
target = tf.placeholder(tf.int32, shape=(None, 20))
target_ = SimpleSparseTensorFrom(target)
loss = tf.nn.ctc_loss(net, target_, sequence_length=[timesteps]*batch_size, time_major=False)
optimizer = tflearn.optimizers.RMSProp(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)

TypeError: 'Tensor' object is not iterable.

In [25]:
import numpy as np

a = np.random.randint(0, 30, (2, 4, 20))
print(a)
print(SimpleSparseTensorFrom(a))

[[[ 3 27  4 14  8 13 17 27 28  6  1 23 22  7  8 29 23 15 20 14]
  [ 0 16 27 12  7 20 25  7  8 10  6 18  8 10 15  3  8  5 25  1]
  [ 7 13 11 22 18  1  8 23  3 16 10 18 20  8 14  9 21 21 22  4]
  [18  9 26 17 19 15 27  5 12  0 29 26 18  6  5 29  1 26  8 20]]

 [[ 5 11 11 20  1 26 14 17 26  4  7 15 18 13 28 16 11 27 14  2]
  [12 11 15 17  3 29  7 15 23 19  1 11  1 16  3  8  9 12  6 25]
  [ 9 23 24 22  8 27 11 12 23 23 25 29  8 23  1  7  6  4  9 19]
  [20 17 16 25 26 20 29  8 15  4  8 24 13  4 14 27 28 21 19  1]]]


TypeError: Expected int32, got array([ 3, 27,  4, 14,  8, 13, 17, 27, 28,  6,  1, 23, 22,  7,  8, 29, 23,
       15, 20, 14]) of type 'ndarray' instead.

In [17]:
np.argmax(a, axis=2)

array([[1, 1, 1],
       [1, 2, 2],
       [0, 2, 0],
       [1, 1, 0],
       [2, 0, 0],
       [1, 2, 1],
       [2, 1, 1],
       [0, 1, 0],
       [0, 0, 1],
       [1, 1, 1]])

In [18]:
for i, v in enumerate(a):
    print(i, v)

(0, array([[23, 24, 24],
       [27, 28, 28],
       [20, 25, 22]]))
(1, array([[22, 28, 23],
       [23, 20, 26],
       [20, 22, 26]]))
(2, array([[28, 26, 28],
       [20, 21, 29],
       [28, 22, 20]]))
(3, array([[20, 29, 29],
       [24, 27, 20],
       [27, 22, 26]]))
(4, array([[20, 21, 24],
       [29, 22, 27],
       [29, 28, 27]]))
(5, array([[22, 27, 23],
       [26, 21, 29],
       [23, 27, 20]]))
(6, array([[21, 22, 25],
       [27, 29, 24],
       [25, 29, 23]]))
(7, array([[25, 21, 22],
       [21, 28, 24],
       [27, 20, 25]]))
(8, array([[29, 20, 23],
       [23, 23, 23],
       [23, 27, 22]]))
(9, array([[20, 29, 27],
       [22, 25, 24],
       [20, 25, 23]]))


In [26]:
def ctc_label_dense_to_sparse(labels, label_lengths):
    # undocumented feature soon to be made public
    from tensorflow.python.ops import functional_ops
    label_shape = tf.shape(labels)
    num_batches_tns = tf.pack([label_shape[0]])
    max_num_labels_tns = tf.pack([label_shape[1]])

    def range_less_than(previous_state, current_input):
        return tf.expand_dims(tf.range(label_shape[1]), 0) < current_input

    init = tf.cast(tf.fill(max_num_labels_tns, 0), tf.bool)
    dense_mask = functional_ops.scan(range_less_than, label_lengths,
                                     initializer=init, parallel_iterations=1)
    dense_mask = dense_mask[:, 0, :]

    label_array = tf.reshape(tf.tile(tf.range(0, label_shape[1]), num_batches_tns),
                             label_shape)
    label_ind = tf.boolean_mask(label_array, dense_mask)

    batch_array = tf.transpose(tf.reshape(tf.tile(tf.range(0, label_shape[0]),
                                                  max_num_labels_tns), tf.reverse(label_shape, [True])))
    batch_ind = tf.boolean_mask(batch_array, dense_mask)
    indices = tf.transpose(tf.reshape(tf.concat(0, [batch_ind, label_ind]), [2,-1]))

    vals_sparse = tf.gather_nd(labels, indices)

    return tf.SparseTensor(tf.to_int64(indices), vals_sparse, tf.to_int64(label_shape))
